# Mount Drive

In [1]:
# mount drive
from os import environ
from google.colab import drive

# if we are starting here we need to remount our drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# Using and Interpreting Categorical Data

In [2]:
import statsmodels.api as sm
from pandas import get_dummies
import seaborn as sns

# Load the tips dataset from seaborn
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [4]:
# One-hot encode the 'sex' categorical variable (drop_first=True to avoid multicollinearity)
df_encoded = get_dummies(df, columns=['sex'], drop_first=True)
df_encoded

,total_bill,tip,smoker,day,time,size,sex_Female
0,16.99,1.01,No,Sun,Dinner,2,1
1,10.34,1.66,No,Sun,Dinner,3,0
2,21.01,3.50,No,Sun,Dinner,3,0
3,23.68,3.31,No,Sun,Dinner,2,0
4,24.59,3.61,No,Sun,Dinner,4,1
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0
240,27.18,2.00,Yes,Sat,Dinner,2,1
241,22.67,2.00,Yes,Sat,Dinner,2,0
242,17.82,1.75,No,Sat,Dinner,2,0


In [5]:
# Prepare data for regression model
X = df_encoded[['total_bill', 'sex_Female']]
X = sm.add_constant(X)  # Add constant term to the predictor
y = df_encoded['tip']

In [6]:
# Train regression model using statsmodels.api
model = sm.OLS(y, X).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                    tip   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     101.3
Date:                Tue, 26 Sep 2023   Prob (F-statistic):           1.18e-32
Time:                        23:37:36   Log-Likelihood:                -350.52
No. Observations:                 244   AIC:                             707.0
Df Residuals:                     241   BIC:                             717.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9067      0.175      5.182      0.0

# True Plots

## Load Data

In [ ]:
# load data
from pandas import read_csv

file_path = "/content/drive/MyDrive/Work/Chapman/MGSC_310/MGSC_310_shared_files_and_resources/Data/mtcars.csv"

cars = read_csv(file_path)

cars.head()

Mounted at /content/drive


,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


# Predict Linear Regression on Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Prepare the data
X = cars.drop(['mpg', 'model'], axis=1)
y = cars['mpg']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=443)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Get predictions for both training and test data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# True Plot Creation

In [ ]:
import altair as alt
from pandas import DataFrame, concat

# Convert the predictions and true values into a dataframe for visualization with Altair
train_data = DataFrame({
    'True Values': y_train,
    'Predicted Values': y_train_pred,
    'Dataset': 'Training'
})

test_data = DataFrame({
    'True Values': y_test,
    'Predicted Values': y_test_pred,
    'Dataset': 'Test'
})

all_data = concat([train_data, test_data])

# Define the perfect prediction line
perfect_line_calc = [min(y) - 1, max(y) + 1]

perfect_prediction = DataFrame({
    'True Values': perfect_line_calc,
    'Predicted Values': perfect_line_calc
})

# Create the Altair plots

# Training data plot
train_chart = alt.Chart(train_data).mark_circle(color='blue').encode(
    x='True Values',
    y='Predicted Values',
    tooltip=['True Values', 'Predicted Values']
).properties(
     width=300,
    height=0,
    title='Predicted Versus True Values (Training Data)'
)

# Test data plot
test_chart = alt.Chart(test_data).mark_circle(color='red').encode(
    x='True Values',
    y='Predicted Values',
    tooltip=['True Values', 'Predicted Values']
).properties(
    width=300,
    height=300,
    title='Predicted Versus True Values (Test Data)'
)

# Perfect prediction line
line_chart = (alt.Chart(perfect_prediction)
                 .mark_line(color='black', strokeDash=[5,5])
                 .encode(
                        x='True Values',
                        y='Predicted Values'
                    )
        )
# Alternative Facet Chart by creating multiple charts
(train_chart + line_chart) | (test_chart + line_chart)


alt.HConcatChart(...)

## Mean Absolute Error (MAE)

 $$
   \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
   $$

MAE represents the average of the absolute differences between the predicted and actual values.

In [ ]:
from typing import List
from numpy import mean
def MAE(true_values: List[float], predicted_values: List[float]) -> float:
  """Calculates the mean absolute error of our prediction. Affected by outliers!"""

  return abs(mean(true_values - predicted_values))

mae_results = MAE(y_test, y_test_pred)
print(f"The MAE of our test predictions is: {mae_results}")

The MAE of our test predictions is: 0.051833015254859206


## Median Absolute Error (MedAE)

  $$
   \text{MedAE} = \text{median}(|y_1 - \hat{y}_1|, |y_2 - \hat{y}_2|, \dots, |y_n - \hat{y}_n|)
   $$


MedAE represents the median of the absolute differences between the predicted and actual values.

In [ ]:
from numpy import median
def MedAE(true_values: List[float], predicted_values: List[float]) -> float:
  """Calculates the median absolute error of our prediction. Less affected by outliers!"""

  return median(abs(true_values - predicted_values))

med_ae_results = MedAE(y_test, y_test_pred)
print(f"The MedAE of our test predictions is: {med_ae_results}")

The MedAE of our test predictions is: 2.057013300407089


# Log-Log Regression Model Coefficients

$$
\log(\text{mpg}_i) = \beta_0 + \beta_1 \log(\text{wt}_i) + \beta_2 \text{disp}_i + \epsilon_i
$$

In [ ]:
from numpy import log

cars['log_mpg'] = log(cars['mpg'])
cars['log_wt'] = log(cars['wt'])

In [ ]:

# Prepare the data
X = cars[['log_wt', 'disp']]
X = sm.add_constant(X)
y = cars['log_mpg']

# Train regression model using statsmodels.api
model = sm.OLS(y, X).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                log_mpg   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     81.64
Date:                Sat, 16 Sep 2023   Prob (F-statistic):           1.22e-12
Time:                        20:34:59   Log-Likelihood:                 24.134
No. Observations:                  32   AIC:                            -42.27
Df Residuals:                      29   BIC:                            -37.87
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7551      0.094     40.090      0.0

# Engineering Non-Linear Regression Models

- Note! You must have both the polynomial and non-polynomial in the regression. Why?

- The linear variable represents the change in the mean of mpg for a one-unit change in horsepower. Horsepower-squared (non-linear variable) represents the rate of change in the slope as horsepower increases.
- In linear regression, always include your polynomial variables up to the highest polynomial. e.g. if you do hp5, you would include hp4, hp3, hp2, and hp.
- This is called **hierarchical model specification.**

In [ ]:
from numpy import log

cars['hp_squared'] = cars['hp']**2

# Prepare the data
X = cars[['log_wt','disp', 'hp', 'hp_squared']]
X = sm.add_constant(X)
y = cars['mpg']

# Train regression model using statsmodels.api
model = sm.OLS(y, X).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     54.39
Date:                Sat, 16 Sep 2023   Prob (F-statistic):           1.57e-12
Time:                        20:59:56   Log-Likelihood:                -67.121
No. Observations:                  32   AIC:                             144.2
Df Residuals:                      27   BIC:                             151.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         42.7420      2.426     17.621      0.0